In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%config IPCompleter.greedy=True

In [2]:
import json
import tweepy
import sys
from tweepy.streaming import StreamListener
import pymongo
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='whitegrid')
from tweepy import OAuthHandler
from tweepy import Stream
import datetime
import csv
import pprint
import pandas as pd

In [3]:
client = pymongo.MongoClient()
db = client['db_tweets_new']
tweets_coll = db['bf_tweets_new']

In [4]:
tweets_coll.find({}).count()

C:\Users\george-lab\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


34084

In [5]:
pprint.pprint(tweets_coll.find()[8]['text'])

('RT @tiffany_caban: .@PeteButtigieg WTF?! Justifying an increase in arrests '
 'of Black men for cannabis in his city by saying it was connected…')


In [6]:
tweets_coll.find({})[2].keys()

dict_keys(['_id', 'created_at', 'id', 'id_str', 'text', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'retweeted_status', 'quoted_status_id', 'quoted_status_id_str', 'quoted_status', 'quoted_status_permalink', 'is_quote_status', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level', 'lang', 'timestamp_ms'])

In [45]:
# Caracteristicas que deseamos del tweet
tweets = []
for result in tweets_coll.find({}):
    tweet_dict = {}
    tweet_dict['id'] = result['id']
    if result['text'][:2]=='RT':
        tweet_dict['text'] = result.get('retweeted_status')
        if tweet_dict['text'] != None:
            tweet_dict['text'] = tweet_dict['text'].get('extended_tweet')
            if tweet_dict['text'] == None:
                tweet_dict['text'] = result['retweeted_status']['text']
            else:
                tweet_dict['text'] = result['retweeted_status']['extended_tweet']['full_text']
    else:
        tweet_dict['text'] = result['text']
    tweet_dict['raw_text'] = result ['text']
    #tweet_dict['coordinates'] = result['coordinates']
    #tweet_dict['token'] = len(result['text'].split())
    #tweet_dict['vocab'] = len(set(result['text'].split()))
    #tweet_dict['vocab/token'] = len(set(result['text'].split()))/len(result['text'].split())
    tweets.append(tweet_dict)
df_tweets = pd.DataFrame(tweets)
df_tweets.shape

(34084, 3)

In [46]:
df_tweets.head()

,id,text,raw_text
0,1226046400175017984,#DemDebate fantastic response by @PeteButtigie...,RT @robertwolf32: #DemDebate fantastic respons...
1,1226046400145715208,@RepAdamSchiff Instead of watching tonight's G...,RT @KashJackson2018: @RepAdamSchiff Instead of...
2,1226046403178164226,How are we supposed to know when @ewarren is t...,RT @jilliansw: How are we supposed to know whe...
3,1226046405610999809,There’s nothing like a bunch of rich white peo...,RT @kimKBaltimore: There’s nothing like a bunc...
4,1226046407372673024,You don't have to assassinate Chris Matthews t...,You don't have to assassinate Chris Matthews t...


In [178]:
df_tweets = df_tweets.dropna()

In [181]:
df_tweets.text.isnull().sum()

0

In [199]:
#df_tweets['@ewarren']=df_tweets.text.apply(lambda x: search_candidates(x,'@ewarren'))
#df_tweets['@PeteButtigieg'] = df_tweets.text.apply(lambda x: search_candidates(x,'@PeteButtigieg'))
#df_tweets['@BernieSanders'] = df_tweets.text.apply(lambda x: search_candidates(x,'@BernieSanders'))
#df_tweets['@JoeBiden'] = df_tweets.text.apply(lambda x: search_candidates(x,'@JoeBiden'))

In [195]:
def search_candidates(text,*words):
    list_of_words = list(words)
    flag = False
    for elem in list_of_words:
        for p in text.split():
            if p == elem:
                flag = True
    return flag

In [76]:
# DICTIONARIO DE LOS ENTES MAS MENCIONADOS
new_dict = {}
for elem in user_mentions_list:
    if elem in new_dict:
        new_dict[elem]=new_dict[elem]+1
    else:
        new_dict[elem]=1
new_dict
sorted(new_dict.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)

[('BernieSanders', 3579),
 ('AndrewYang', 1656),
 ('PeteButtigieg', 1617),
 ('JoeBiden', 1382),
 ('KimStrassel', 1166),
 ('ewarren', 1153),
 ('ABC', 836),
 ('KyleKulinski', 745),
 ('TrumpWarRoom', 676),
 ('realDonaldTrump', 671),
 ('ninaturner', 635),
 ('itsafshar', 601),
 ('GOPChairwoman', 530),
 ('Harvard4Bernie', 480),
 ('briebriejoy', 472),
 ('People4Bernie', 468),
 ('sunrisemvmt', 348),
 ('DiamondandSilk', 337),
 ('johniadarola', 321),
 ('HardballChris', 317),
 ('amyklobuchar', 306),
 ('AmericaFirstPAC', 293),
 ('BernForBernie20', 252),
 ('GOP', 248),
 ('daviddoel', 214),
 ('krystalball', 201),
 ('ProudResister', 200),
 ('ScottPresler', 195),
 ('Bern2Bern', 195),
 ('JayarJackson', 192),
 ('kayleighmcenany', 190),
 ('tiffany_caban', 189),
 ('nickisnixed', 189),
 ('joncoopertweets', 187),
 ('cmclymer', 187),
 ('jimmy_dore', 184),
 ('mmpadellan', 177),
 ('LukewSavage', 164),
 ('AaronStigile', 159),
 ('ABCPolitics', 157),
 ('RealTimBlack', 148),
 ('ZephyrTeachout', 146),
 ('DonaldJTru

In [254]:
# DEPURAMOS EL DATASET POR MENCIONADOS
ewarren_list = df_tweets.text.apply(lambda x: search_candidates(x,'@ewarren')).values
buttig_list = df_tweets.text.apply(lambda x: search_candidates(x,'@PeteButtigieg')).values
bernie_list = df_tweets.text.apply(lambda x: search_candidates(x,'@BernieSanders','@Harvard4Bernie','@People4Bernie','@BernForBernie20','bernie','sanders')).values
biden_list = df_tweets.text.apply(lambda x: search_candidates(x,'@JoeBiden')).values
andrew_list = df_tweets.text.apply(lambda x: search_candidates(x,'@AndrewYang')).values
trump_list = df_tweets.text.apply(lambda x: search_candidates(x,'@TrumpWarRoom','@realDonaldTrump')).values

In [238]:
kim_list.value_counts()

False    34041
True         2
Name: raw_text, dtype: int64

In [224]:
df_tweets.text.apply(lambda x: search_candidates(x,'@KimStrassel','@ewarren')).values

array([False, False,  True, ..., False, False, False])

In [248]:
df_tweets.iloc[trump_list].iloc[0].text

'#DemDebate moderator: What will you do about child poverty?\n\nElect @realDonaldTrump should be the answer because...\n\nAlmost 650,000 children raised by single moms have risen out of poverty under his leadership!'

In [260]:
df_tweets.iloc[trump_list].shape

(897, 4)

In [184]:
df_tweets

,id,text,raw_text,@ewarren
0,1226046400175017984,#DemDebate fantastic response by @PeteButtigie...,RT @robertwolf32: #DemDebate fantastic respons...,False
1,1226046400145715208,@RepAdamSchiff Instead of watching tonight's G...,RT @KashJackson2018: @RepAdamSchiff Instead of...,False
2,1226046403178164226,How are we supposed to know when @ewarren is t...,RT @jilliansw: How are we supposed to know whe...,True
3,1226046405610999809,There’s nothing like a bunch of rich white peo...,RT @kimKBaltimore: There’s nothing like a bunc...,False
4,1226046407372673024,You don't have to assassinate Chris Matthews t...,You don't have to assassinate Chris Matthews t...,False
...,...,...,...,...
34079,1226096598188322816,"Um, ouch, that stung. #DemDebate https://t.co/...","RT @Pa1Lauren: Um, ouch, that stung. #DemDebat...",False
34080,1226096600784756736,2020 Democrats should be panicking.\n\nIn Iowa...,RT @GOPChairwoman: 2020 Democrats should be pa...,False
34081,1226096601397170176,Ummm. Why is #chrischristie offering input on ...,RT @TiffanyDCross: Ummm. Why is #chrischristie...,False
34082,1226096602378579968,"As a political journalist, my editors taught m...","As a political journalist, my editors taught m...",False


In [174]:
search_candidates(df_tweets,'he')

False

In [19]:
tweets_coll.find({})[2].keys()

dict_keys(['_id', 'created_at', 'id', 'id_str', 'text', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'retweeted_status', 'quoted_status_id', 'quoted_status_id_str', 'quoted_status', 'quoted_status_permalink', 'is_quote_status', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level', 'lang', 'timestamp_ms'])

In [63]:
tweets_coll.find({})[8]['id']

1226046410547748864

In [31]:
df_tweets.coordinates.isnull().sum()

34084

In [65]:
user_mentions_list=[]
for tweets in tweets_coll.find({}):
    for entities in tweets['entities']['user_mentions']:
        user_mentions_list.append(entities['screen_name'])
user_mentions_list

['robertwolf32',
 'PeteButtigieg',
 'JoeBiden',
 'KashJackson2018',
 'RepAdamSchiff',
 'jilliansw',
 'ewarren',
 'kimKBaltimore',
 'NationalNurses',
 'SunriseSci',
 'briebriejoy',
 'tiffany_caban',
 'PeteButtigieg',
 'troyrudd',
 'itsafshar',
 'BernieSanders',
 'don_diver',
 'krystalball',
 'TulsiGabbard',
 'BernieSanders',
 'StyleWriterNYC',
 'AndrewYang',
 'KyleKulinski',
 'osPatriot',
 'ABC',
 'kayleighmcenany',
 'realDonaldTrump',
 'KyleKulinski',
 'stillgray',
 'JordanChariton',
 'AndrewHClark',
 'GOPChairwoman',
 'GOP',
 'KimStrassel',
 'godzillo_',
 'NasshanPassion',
 'AndrewYang',
 'GiffordsCourage',
 'ewarren',
 'thehill',
 'JonathanCohn',
 'realmikefox',
 'holyspacemonkey',
 'JayPotato1',
 'AndrewYang',
 'SunriseSci',
 'realmikefox',
 'joncoopertweets',
 'JoeBiden',
 'NationalNurses',
 'SarahAMatthews1',
 'KassyDillon',
 'LoDov_',
 'BernForBernie20',
 'TomPerez',
 'cmclymer',
 'ewarren',
 'FrankLuntz',
 'BernieSanders',
 'ZaidJilani',
 'ACTBrigitte',
 'Solo77203564',
 'IdahoD

In [128]:
def select_paragraph(text, word, delimiter='\n'):
    return [p for p in text.split(delimiter) if word in p]

In [127]:
select

In [112]:
N_indices = list(N_indices)

In [126]:
N_indices

[1,
 12,
 63,
 71,
 105,
 121,
 132,
 165,
 221,
 224,
 235,
 252,
 264,
 273,
 284,
 322,
 341,
 363,
 366,
 375,
 397,
 404,
 434,
 464,
 470,
 472,
 482,
 495,
 538,
 572,
 668,
 697,
 698,
 716,
 734,
 748,
 788,
 837,
 844,
 876,
 889,
 924,
 931,
 977,
 993,
 1007,
 1037,
 1052,
 1075,
 1082,
 1086,
 1090,
 1097,
 1114,
 1130,
 1202,
 1216,
 1234,
 1246,
 1252,
 1341,
 1387,
 1403,
 1419,
 1421,
 1440,
 1474,
 1504,
 1513,
 1516,
 1555,
 1564,
 1608,
 1627,
 1630,
 1672,
 1708,
 1717,
 1738,
 1747,
 1793,
 1847,
 1897,
 1912,
 1920,
 1924,
 1940,
 1951,
 1955,
 1960,
 1978,
 1997,
 1999,
 2023,
 2051,
 2054,
 2073,
 2098,
 2109,
 2115,
 2117,
 2124,
 2146,
 2154,
 2157,
 2198,
 2224,
 2257,
 2264,
 2273,
 2297,
 2304,
 2314,
 2361,
 2378,
 2394,
 2409,
 2429,
 2439,
 2445,
 2463,
 2475,
 2483,
 2541,
 2555,
 2616,
 2632,
 2639,
 2722,
 2749,
 2759,
 2789,
 2794,
 2835,
 2838,
 2843,
 2846,
 2854,
 2873,
 2895,
 2904,
 2906,
 2946,
 2954,
 2966,
 2974,
 3093,
 3096,
 3139,
 3145,
